In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-community
!pip install langchain_openai
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.8 MB/s eta 0:00:00


DOCUMENT LOADER

In [ ]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path='/content/drive/MyDrive/Final_Project/ai-medical-chatbot_500_Doctor.csv')
data = loader.load()

In [ ]:
print(data)

Buffered data was truncated after reaching the output size limit.

In [ ]:
loader_mediAI = CSVLoader(file_path='/content/drive/MyDrive/Final_Project/ai-medical-chatbot_500.csv', source_column="Doctor")

Doctor_data = loader_mediAI.load()

TEXT SPLITTER (RECURSIVELY SPLIT BY CHARACTER)


In [ ]:
with open("/content/drive/MyDrive/Final_Project/ai-medical-chatbot_500_Doctor.csv") as f:
    Medical_Dr_Note_ORG = f.read()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
Dr_texts = text_splitter.create_documents([Medical_Dr_Note_ORG])
#print(Dr_texts[0])
#print(Dr_texts[1])

In [ ]:
print(Dr_texts[0])

page_content='Doctor
Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'


In [ ]:
len(Dr_texts)

1099

EMBEDDINGS MODEL

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('/content/drive/MyDrive/Final_Project/OpenAI.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#print(f"OPENAI_API Key: {OPENAI_API_KEY}")

In [ ]:
#embed the chunks of text
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

underlying_embeddings = OpenAIEmbeddings()
store = LocalFileStore("./cache/")

#cached embeddings so we don't have to do it again
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings,
    store,
    namespace=underlying_embeddings.model
)


In [ ]:
list(store.yield_keys())

[]

VECTOR DOCUMENTS USING Facebook AI Similarity Search (FAISS) library

In [ ]:
#!pip install faiss-gpu

#%%time
db = FAISS.from_documents(Dr_texts, cached_embedder)

In [ ]:
#save faiss index
#Python pickle file that stores metadata about the indexed documents
# faiss_index binary file that stores the actual FAISS index, which means:
#The numerical vectors (embeddings)

#save faiss index
db.save_local('/content/drive/MyDrive/Final_Project/my_faiss_index')

In [ ]:
query = "small cysts are concern"

docs = db.similarity_search(query, k=3)
print(docs[0])

page_content='"Hello. I understand your concern. The thing on the chest looks more of a sebaceous cyst, (attachment removed to protect patient identity) which comes and goes even if you pop out the contents as it needs to be excised completely to prevent the recurrence. For the toenail spot, I am not sure that'


In [ ]:
### Gnerative question answering
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI # Import ChatOpenAI

llm_model = ChatOpenAI(openai_api_key= OPENAI_API_KEY, temperature=0.0)

query2 = "what type of small cysts are concern"
qa = RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever=db.as_retriever(),
    return_source_documents=True
)

<ipython-input-26-e769c266b269>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model = ChatOpenAI(openai_api_key= OPENAI_API_KEY, temperature=0.0)


In [ ]:
qa.invoke(query2)

{'query': 'small cysts are concern',
 'result': "Yes, small cysts can sometimes be a concern, especially if they are recurring or causing discomfort. It's important to have them evaluated by a healthcare professional to determine the best course of action.",
 'source_documents': [Document(id='b06cd1bd-a77d-4653-964b-902f3b2496ff', metadata={}, page_content='"Hello. I understand your concern. The thing on the chest looks more of a sebaceous cyst, (attachment removed to protect patient identity) which comes and goes even if you pop out the contents as it needs to be excised completely to prevent the recurrence. For the toenail spot, I am not sure that'),
  Document(id='f325c49c-462f-47f1-8fcf-e4683d45e9ca', metadata={}, page_content='"Hello. Small spot in center of forehead is seen in some pictures (attachment removed to protect patient identity). As you know skull is made up of 16 small bones, these bones are attached with cartilages and get ossified with times. Due to some gaping, this

##AGENT

In [ ]:
#import dependencies for agent to work
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI


In [ ]:
def init_faiss():
    # Load or build your FAISS index
    vectorstore = FAISS.load_local("/content/drive/MyDrive/Final_Project/my_faiss_index/index.faiss", embeddings=cached_embbeder)
    return vectorstore

In [ ]:
#chat model
chat_model = ChatOpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a helpful Medical AI bot. Your name is MediChat. Please help them to answer
        any medical questions they have"""),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)